# test siu nhanh cái model (trong colab)

target: death, injure, total dmg adjust . metric: mse, terminate the large error in predict dmg, phu hop voi gbms

In [24]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [39]:
df = pd.read_csv('/content/STORM_RAW_1.csv')

In [40]:
df

,Unnamed: 0,ID,EventName,Year,Month,DisasterType,DisasterSubtype,MainLandfallLocation,Flood,Slide,OFDAResponse,Appeal,Declaration,LandfallMagnitude(kph),LandfallPressure(mb),TotalDeaths,NoInjured,TotalDamage(000US$),TotalDamageAdjusted(000US$),CPI
0,0,STORM_NAN_1953,NaN,1953,9.0,Storm,Tropical cyclone,Mekong River Delta,0.0,0.0,0.0,0.0,0.0,93.0,985.0,1000.0,146.0,89730.0,139143.00,9.156133
1,1,STORM_VIOLET_1964,Violet,1964,9.0,Storm,Tropical cyclone,North Central Coast,NaN,NaN,0.0,0.0,0.0,93.0,985.0,124.0,146.0,10000.0,94354.08,10.598376
2,2,STORM_IRIS_1964,Iris,1964,11.0,Storm,Tropical cyclone,South Central Coast,NaN,NaN,0.0,0.0,0.0,150.0,960.0,5100.0,146.0,70000.0,660478.55,10.598376
3,3,STORM_JOAN_1964,Joan,1964,11.0,Storm,Tropical cyclone,South Central Coast,NaN,NaN,0.0,0.0,0.0,130.0,980.0,2500.0,146.0,15000.0,141531.12,10.598376
4,4,STORM_KATE_1964,Kate,1964,11.0,Storm,Tropical cyclone,Central Highlands,NaN,NaN,0.0,0.0,0.0,130.0,970.0,0.0,146.0,3000.0,28306.22,10.598376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,179,STORM_KOMPASU_2021,Kompasu,2021,10.0,Storm,Tropical cyclone,North Central Coast,0.0,0.0,0.0,0.0,0.0,46.0,1002.0,124.0,146.0,89730.0,139143.00,92.590193
180,180,STORM_NAN_2022,NaN,2022,5.0,Storm,Storm (General),NorthEast,1.0,1.0,0.0,0.0,0.0,93.0,985.0,9.0,146.0,89730.0,139143.00,100.000000
181,181,STORM_NORU(KARDING)_2022,Noru(Karding),2022,9.0,Storm,Storm surge,South Central Coast,1.0,0.0,0.0,0.0,0.0,176.0,949.0,9.0,62.0,108300.0,108300.00,100.000000
182,182,STORM_SONCA_2022,Sonca,2022,10.0,Storm,Tropical cyclone,South Central Coast,1.0,1.0,0.0,0.0,0.0,55.0,1000.0,19.0,110.0,65000.0,65000.00,100.000000


In [41]:
# prompt: delete col 	OFDAResponse	Appeal	Declaration and CPI

df = df.drop(['OFDAResponse', 'Appeal', 'Declaration','CPI'], axis=1)


In [42]:
df['Flood'] = df['Flood'].fillna(0)
df['Slide'] = df['Slide'].fillna(0)

## test one hot encoding

In [43]:
# Step 1: Create a mapping for the specific landfall labels
landfall_mapping = {
    'Đông Bắc Bộ': 1,
    'Tây Bắc Bộ': 2,
    'Đồng Bằng Sông Hồng (Hà Nội)': 3,
    'Bắc Trung Bộ': 4,
    'Nam Trung Bộ (Đà Nẵng)': 5,
    'Tây Nguyên': 6,
    'Đông Nam Bộ (TPHCM)': 7,
    'Đồng bằng sông cửu long': 8
}

# Step 2: Map the 'MainLandfallLocation' column to these numerical values
df['MainLandfallLocation'] = df['MainLandfallLocation'].map(landfall_mapping)

# Step 3: Apply one-hot encoding for remaining categorical columns like DisasterType and DisasterSubtype
categorical_cols = ['DisasterType', 'DisasterSubtype']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Step 4: Standardize numerical columns LandfallMagnitude(kph) and LandfallPressure(mb)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['LandfallMagnitude(kph)', 'LandfallPressure(mb)']] = scaler.fit_transform(df[['LandfallMagnitude(kph)', 'LandfallPressure(mb)']])

# Display the processed DataFrame to verify changes
df.head()


,Unnamed: 0,ID,EventName,Year,Month,MainLandfallLocation,Flood,Slide,LandfallMagnitude(kph),LandfallPressure(mb),...,TotalDamage(000US$),TotalDamageAdjusted(000US$),DisasterSubtype_Hail,DisasterSubtype_Lightning/Thunderstorms,DisasterSubtype_Severe weather,DisasterSubtype_Storm (General),DisasterSubtype_Storm surge,DisasterSubtype_Tornado,DisasterSubtype_Tropical cyclone,DisasterSubtype_Tropical depression
0,0,STORM_NAN_1953,NaN,1953,9.0,NaN,0.0,0.0,0.008079,0.009049,...,89730.0,139143.00,False,False,False,False,False,False,True,False
1,1,STORM_VIOLET_1964,Violet,1964,9.0,NaN,0.0,0.0,0.008079,0.009049,...,10000.0,94354.08,False,False,False,False,False,False,True,False
2,2,STORM_IRIS_1964,Iris,1964,11.0,NaN,0.0,0.0,1.606848,-1.656017,...,70000.0,660478.55,False,False,False,False,False,False,True,False
3,3,STORM_JOAN_1964,Joan,1964,11.0,NaN,0.0,0.0,1.045876,-0.323964,...,15000.0,141531.12,False,False,False,False,False,False,True,False
4,4,STORM_KATE_1964,Kate,1964,11.0,NaN,0.0,0.0,1.045876,-0.989991,...,3000.0,28306.22,False,False,False,False,False,False,True,False


from matplotlib import pyplot as plt
_df_23['Unnamed: 0'].plot(kind='hist', bins=20, title='Unnamed: 0')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_24['Year'].plot(kind='hist', bins=20, title='Year')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_25['Month'].plot(kind='hist', bins=20, title='Month')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_26['LandfallMagnitude(kph)'].plot(kind='hist', bins=20, title='LandfallMagnitude(kph)')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_43['EventName'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_43, x='Unnamed: 0', y='EventName', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_44['ID'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_44, x='Year', y='ID', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_45['EventName'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_45, x='Year', y='EventName', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

## XGBOOST

In [45]:
predictors = ['LandfallMagnitude(kph)', 'LandfallPressure(mb)', 'Flood', 'Slide', 'Year', 'Month'] + \
             [col for col in df.columns if col.startswith('MainLandfallLocation_') or col.startswith('DisasterSubtype_')]

# Exclude highly correlated variables like TotalDamage and focus on TotalDamageAdjusted
target = 'TotalDamageAdjusted(000US$)'

X = df[predictors]
y = df[target]

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(random_state=42)

In [47]:
param_grid = {
    'n_estimators': [100],
    'learning_rate': [0.1],
    'max_depth': [3, 5],
    'subsample': [1.0]
}

# Set up cross-validation with grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)

# Step 4: Train the model using cross-validation to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Step 5: Use the best found model to make predictions
best_model = grid_search.best_estimator_

y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# Step 6: Evaluate the model using MSE for both training and testing sets
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

# Step 7: Analyze feature importance from the best model
feature_importance = best_model.feature_importances_

# Create a DataFrame to display feature importance alongside their names
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

# Output the results: MSE and feature importance
mse_train, mse_test, grid_search.best_params_, importance_df.head(10)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


(4972351760.12769,
 57972138498.95392,
 {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0},
                                 Feature  Importance
 0                LandfallMagnitude(kph)    0.293669
 4                                  Year    0.230933
 3                                 Slide    0.105127
 1                  LandfallPressure(mb)    0.094024
 2                                 Flood    0.092229
 5                                 Month    0.092033
 13  DisasterSubtype_Tropical depression    0.067468
 12     DisasterSubtype_Tropical cyclone    0.013063
 9       DisasterSubtype_Storm (General)    0.011454
 6                  DisasterSubtype_Hail    0.000000)

model overfitting , sẽ sửa ratio train test,  LandfallMagnitude là feature quan trọng nhất, Year cũng có tác động rất lớn vào thiệt hại, flood và slide đồng thời cũng có tác dụng lớn. DisasterSubtype k quan trọng lắm.

## LightGBM , test sau